In [1]:
import joblib

In [2]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from xgboost  import XGBRegressor

In [3]:
import pandas as pd
import numpy as np 
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline
import warnings # Filtering warnings
warnings.filterwarnings('ignore')




In [4]:
df

,STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,PRCP,PRCP_ATTRIBUTES,TAVG,TAVG_ATTRIBUTES,TMAX,TMAX_ATTRIBUTES,TMIN,TMIN_ATTRIBUTES
0,IN022021900,"NEW DELHI SAFDARJUN, IN",28.583,77.2,216.0,2000-01-01,0.0,"D,,S",11.1,"H,,S",19.5,",,S",5.6,",,S"
1,IN022021900,"NEW DELHI SAFDARJUN, IN",28.583,77.2,216.0,2000-01-02,0.0,"D,,S",10.6,"H,,S",18.8,",,S",7.7,",,S"
2,IN022021900,"NEW DELHI SAFDARJUN, IN",28.583,77.2,216.0,2000-01-03,0.0,"D,,S",9.4,"H,,S",19.2,",,S",6.6,",,S"
3,IN022021900,"NEW DELHI SAFDARJUN, IN",28.583,77.2,216.0,2000-01-04,0.0,"D,,S",8.7,"H,,S",NaN,NaN,NaN,NaN
4,IN022021900,"NEW DELHI SAFDARJUN, IN",28.583,77.2,216.0,2000-01-05,0.0,"D,,S",9.5,"H,,S",15.8,",,S",5.0,",,S"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8008,IN022021900,"NEW DELHI SAFDARJUN, IN",28.583,77.2,216.0,2021-12-28,0.0,",,S",12.9,"H,,S",22.6,",,S",9.4,",,S"
8009,IN022021900,"NEW DELHI SAFDARJUN, IN",28.583,77.2,216.0,2021-12-29,4.1,",,S",13.3,"H,,S",NaN,NaN,8.4,",,S"
8010,IN022021900,"NEW DELHI SAFDARJUN, IN",28.583,77.2,216.0,2021-12-30,0.0,",,S",11.0,"H,,S",21.2,",,S",3.4,",,S"
8011,IN022021900,"NEW DELHI SAFDARJUN, IN",28.583,77.2,216.0,2021-12-31,NaN,NaN,10.7,"H,,S",19.5,",,S",3.8,",,S"


In [5]:
df = df.drop(['STATION' ,'NAME' , 'PRCP_ATTRIBUTES' , 'TAVG_ATTRIBUTES' , 'TMAX_ATTRIBUTES' , 'TMIN_ATTRIBUTES' ], axis =1)

In [6]:
df = df.drop(['ELEVATION'], axis = 1)
df

,LATITUDE,LONGITUDE,DATE,PRCP,TAVG,TMAX,TMIN
0,28.583,77.2,2000-01-01,0.0,11.1,19.5,5.6
1,28.583,77.2,2000-01-02,0.0,10.6,18.8,7.7
2,28.583,77.2,2000-01-03,0.0,9.4,19.2,6.6
3,28.583,77.2,2000-01-04,0.0,8.7,NaN,NaN
4,28.583,77.2,2000-01-05,0.0,9.5,15.8,5.0
...,...,...,...,...,...,...,...
8008,28.583,77.2,2021-12-28,0.0,12.9,22.6,9.4
8009,28.583,77.2,2021-12-29,4.1,13.3,NaN,8.4
8010,28.583,77.2,2021-12-30,0.0,11.0,21.2,3.4
8011,28.583,77.2,2021-12-31,NaN,10.7,19.5,3.8


In [7]:
df.describe()

,LATITUDE,LONGITUDE,PRCP,TAVG,TMAX,TMIN
count,8.013000e+03,8.013000e+03,2564.000000,8013.000000,6917.000000,6975.000000
mean,2.858300e+01,7.720000e+01,5.333463,25.145638,31.898251,18.805147
std,1.421174e-14,2.842348e-14,13.984311,7.337582,6.805178,7.757557
min,2.858300e+01,7.720000e+01,0.000000,6.600000,9.800000,0.200000
25%,2.858300e+01,7.720000e+01,0.000000,18.600000,26.900000,11.800000
50%,2.858300e+01,7.720000e+01,0.000000,27.200000,33.400000,19.900000
75%,2.858300e+01,7.720000e+01,3.000000,31.100000,36.700000,25.800000
max,2.858300e+01,7.720000e+01,166.900000,39.800000,46.000000,34.200000


In [9]:
df.apply(pd.isnull).sum()/df.shape[0]

LATITUDE     0.000000
LONGITUDE    0.000000
DATE         0.000000
PRCP         0.680020
TAVG         0.000000
TMAX         0.136778
TMIN         0.129539
dtype: float64

In [10]:
df = df.drop(['PRCP'], axis =1)
df

,LATITUDE,LONGITUDE,DATE,TAVG,TMAX,TMIN
0,28.583,77.2,2000-01-01,11.1,19.5,5.6
1,28.583,77.2,2000-01-02,10.6,18.8,7.7
2,28.583,77.2,2000-01-03,9.4,19.2,6.6
3,28.583,77.2,2000-01-04,8.7,NaN,NaN
4,28.583,77.2,2000-01-05,9.5,15.8,5.0
...,...,...,...,...,...,...
8008,28.583,77.2,2021-12-28,12.9,22.6,9.4
8009,28.583,77.2,2021-12-29,13.3,NaN,8.4
8010,28.583,77.2,2021-12-30,11.0,21.2,3.4
8011,28.583,77.2,2021-12-31,10.7,19.5,3.8


In [11]:
df = df.drop(['LATITUDE' , 'LONGITUDE'], axis =1)
df

,DATE,TAVG,TMAX,TMIN
0,2000-01-01,11.1,19.5,5.6
1,2000-01-02,10.6,18.8,7.7
2,2000-01-03,9.4,19.2,6.6
3,2000-01-04,8.7,NaN,NaN
4,2000-01-05,9.5,15.8,5.0
...,...,...,...,...
8008,2021-12-28,12.9,22.6,9.4
8009,2021-12-29,13.3,NaN,8.4
8010,2021-12-30,11.0,21.2,3.4
8011,2021-12-31,10.7,19.5,3.8


In [12]:
df.corr()

,TAVG,TMAX,TMIN
TAVG,1.000000,0.960229,0.963415
TMAX,0.960229,1.000000,0.886249
TMIN,0.963415,0.886249,1.000000


In [13]:
# Converting Date datatypes and spliting date into date, month and year.
df['DATE']=pd.to_datetime(df['DATE'])
df['Day']=df['DATE'].apply(lambda x:x.day)
df['Month']=df['DATE'].apply(lambda x:x.month)
df['Year']=df['DATE'].apply(lambda x:x.year)
df.head()

,DATE,TAVG,TMAX,TMIN,Day,Month,Year
0,2000-01-01,11.1,19.5,5.6,1,1,2000
1,2000-01-02,10.6,18.8,7.7,2,1,2000
2,2000-01-03,9.4,19.2,6.6,3,1,2000
3,2000-01-04,8.7,NaN,NaN,4,1,2000
4,2000-01-05,9.5,15.8,5.0,5,1,2000


In [14]:
df = df.drop(['DATE'] , axis =1)

In [15]:
df.head()

,TAVG,TMAX,TMIN,Day,Month,Year
0,11.1,19.5,5.6,1,1,2000
1,10.6,18.8,7.7,2,1,2000
2,9.4,19.2,6.6,3,1,2000
3,8.7,NaN,NaN,4,1,2000
4,9.5,15.8,5.0,5,1,2000


In [16]:
df.corr()

,TAVG,TMAX,TMIN,Day,Month,Year
TAVG,1.000000,0.960229,0.963415,0.001541,0.070065,0.033605
TMAX,0.960229,1.000000,0.886249,0.004968,0.072023,0.034898
TMIN,0.963415,0.886249,1.000000,-0.003718,0.099878,-0.009569
Day,0.001541,0.004968,-0.003718,1.000000,0.011116,-0.000627
Month,0.070065,0.072023,0.099878,0.011116,1.000000,0.001362
Year,0.033605,0.034898,-0.009569,-0.000627,0.001362,1.000000


TAVG MODEL

In [17]:
# Splitting data in target and dependent feature
X = df.drop(['TAVG'], axis =1)
Y = df['TAVG']

In [18]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
X_scale = scaler.fit_transform(X)

In [19]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=42, test_size=.33)

In [20]:
print('Training feature matrix size:',X_train.shape)
print('Training target vector size:',Y_train.shape)
print('Test feature matrix size:',X_test.shape)
print('Test target vector size:',Y_test.shape)

Training feature matrix size: (5368, 5)
Training target vector size: (5368,)
Test feature matrix size: (2645, 5)
Test target vector size: (2645,)


In [21]:
from sklearn.model_selection import GridSearchCV

gridsearch

In [22]:
parameter = {'n_estimators':[200,400,500,600],'gamma':np.arange(0,0.2,0.1),
              'booster' : ['gbtree','dart','gblinear'], 'max_depth':[4,6,8,10],
              'eta' : [0.001, 0.01, 0.1] }

In [51]:
GCV = GridSearchCV(XGBRegressor(),parameter,verbose =10)

RUN Grid Search

In [64]:
GCV.best_params_

AttributeError: 'GridSearchCV' object has no attribute 'best_params_'

In [23]:
Final_mod =  XGBRegressor(n_estimators=100 , booster= 'dart', eta= 0.1, max_depth= 10, gamma= 0.0 )
Final_mod.fit(X_train,Y_train)
y_pred=Final_mod.predict(X_test)

In [24]:
print('Mean absolute error :', mean_absolute_error(Y_test,y_pred))
print('Mean squared error :', mean_squared_error(Y_test,y_pred))
print('Root Mean Squared Error:', np.sqrt(mean_squared_error(Y_test,y_pred)))
print('R2 error :', r2_score(Y_test,y_pred)) 

Mean absolute error : 0.7523784645383436
Mean squared error : 1.0908700840364063
Root Mean Squared Error: 1.0444472624486152
R2 error : 0.9792352588391006


In [25]:
joblib.dump(Final_mod,'TAVG_Prediction.pkl')

['TAVG_Prediction.pkl']

In [30]:
actual = np.array(Y_test)
predicted = np.array(Final_mod.predict(X_test))
df_Predicted = pd.DataFrame({"Actual Values":actual,"Predicted Values":predicted},index= range(len(actual)))
df_Predicted 

,Actual Values,Predicted Values
0,15.8,15.887500
1,28.7,29.020086
2,22.3,23.271784
3,34.4,34.355888
4,23.9,24.606831
...,...,...
2640,15.9,15.832867
2641,28.1,29.413336
2642,30.2,28.433607
2643,29.9,30.826885


TMAX Model

In [27]:
# Splitting data in target and dependent feature
K = df.drop(['TMAX'], axis =1)
L = df['TMAX']

In [31]:
from sklearn.preprocessing import StandardScaler
scaler= StandardScaler()
K_scale = scaler.fit_transform(K)

In [32]:
K_train, K_test, L_train, L_test = train_test_split(K, L, random_state=42, test_size=.33)

In [33]:
print('Training feature matrix size:',K_train.shape)
print('Training target vector size:',L_train.shape)
print('Test feature matrix size:',K_test.shape)
print('Test target vector size:',L_test.shape)

Training feature matrix size: (5368, 5)
Training target vector size: (5368,)
Test feature matrix size: (2645, 5)
Test target vector size: (2645,)


In [34]:
K_test

,TAVG,TMIN,Day,Month,Year
1420,15.8,8.1,23,11,2003
1728,28.7,22.2,26,9,2004
6615,22.3,15.4,27,2,2018
4507,34.4,28.0,18,5,2012
3362,23.9,18.9,21,3,2009
...,...,...,...,...,...
4750,15.9,8.8,16,1,2013
2431,28.1,23.3,30,8,2006
3739,30.2,20.7,2,4,2010
564,29.9,28.3,19,7,2001


In [37]:
Final_mod1 = XGBRegressor(n_estimators=100 , booster= 'dart', eta= 0.1, max_depth= 10, gamma= 0.0 )
Final_mod1.fit(K_train,L_train)
L_pred=Final_mod1.predict(K_test)

XGBoostError: [10:25:04] /Users/runner/work/xgboost/xgboost/python-package/build/temp.macosx-10.9-x86_64-cpython-37/xgboost/src/data/data.cc:458: Check failed: valid: Label contains NaN, infinity or a value too large.
Stack trace:
  [bt] (0) 1   libxgboost.dylib                    0x000000012fa97705 dmlc::LogMessageFatal::~LogMessageFatal() + 117
  [bt] (1) 2   libxgboost.dylib                    0x000000012faf2de7 xgboost::MetaInfo::SetInfoFromHost(xgboost::GenericParameter const&, xgboost::StringView, xgboost::Json) + 5415
  [bt] (2) 3   libxgboost.dylib                    0x000000012faf17d2 xgboost::MetaInfo::SetInfo(xgboost::GenericParameter const&, xgboost::StringView, xgboost::StringView) + 146
  [bt] (3) 4   libxgboost.dylib                    0x000000012fa98acf XGDMatrixSetInfoFromInterface + 223
  [bt] (4) 5   libffi.dylib                        0x00007ff81ee7f882 ffi_call_unix64 + 82
  [bt] (5) 6   ???                                 0x00007ff7b5807860 0x0 + 140701878745184



In [35]:
joblib.dump(Final_mod1,'TMAX_Prediction.pkl')

NameError: name 'Final_mod1' is not defined